# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**: Structured Streaming with Files Description

**Fecha**:28/03/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Structured-Streaming") \
    .master("spark://b631371550f9:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/04 14:31:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Checa el pwd

In [3]:
!pwd

/home/jovyan


### Generate logs

In [4]:
from team_bellaco.log_gen import generate_logs
generate_logs(100,"/home/jovyan/notebooks/data/logs","log")

Logs guardados en: /home/jovyan/notebooks/data/logs/log-99.log


Registra el Listener del Team Bellaco

In [5]:
from team_bellaco.traffic_query_listener import TrafficListener
spark.streams.addListener(TrafficListener())

Lee los logs como stream

In [6]:
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import split, col

log_schema = StructType().add("raw", StringType())

log_df = (
    spark.readStream.format("text")
    .schema(log_schema)
    .load("/home/jovyan/notebooks/data/logs/")
)

Separa las columnas de los logs (timestamp, level, message, server)

In [7]:
parsed_df = (
    log_df.withColumn("parts", split(col("raw"), " \\| "))
    .withColumn("timestamp", col("parts").getItem(0))
    .withColumn("level", col("parts").getItem(1))
    .withColumn("message", col("parts").getItem(2))
    .withColumn("server", col("parts").getItem(3))
    .drop("parts")
)

Filtra los errores 500 de los logs

In [10]:
errors_df = parsed_df.filter(col("message").contains("500 Internal Server Error"))

Se ejecuta el stream y se detiene manualmente con 30 segundos 

In [12]:
query = (
    errors_df.writeStream.outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

query.awaitTermination(30)

25/04/04 14:34:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-98837ea7-f455-4f6e-868d-0a8d3d1d99e2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/04 14:34:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: 25079c4f-92b4-4b34-8f53-5bd10aa9342d


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|raw                                                                    |timestamp          |level|message                  |server       |
+-----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-3|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-3|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-3|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-3|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-1|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-1|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | serv

False

In [13]:
# Stop the SparkContext
sc.stop()